# Crypto ZigZag ML - GPU Training on Google Colab

This notebook runs on Google Colab with free GPU support.

Instructions:
1. Open this notebook in Google Colab
2. Run all cells in order
3. GPU will be enabled automatically


## Step 1: Setup and Mount Google Drive


In [ ]:
# Check GPU availability
import tensorflow as tf

print(f'GPU Available: {len(tf.config.list_physical_devices("GPU"))}')
print(f'GPU Devices: {tf.config.list_physical_devices("GPU")}')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository if needed
!cd /content/drive/MyDrive && git clone https://github.com/caizongxun/crypto-zigzag-ml.git
!cd /content/drive/MyDrive/crypto-zigzag-ml && pip install -q -r requirements.txt

## Step 2: Import Libraries and Setup Paths


In [ ]:
import sys
from pathlib import Path

# Add project to path
project_root = '/content/drive/MyDrive/crypto-zigzag-ml'
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from data.fetch_data import CryptoDataFetcher
from src.zigzag_indicator import ZigZagIndicator
from src.features import FeatureEngineer
from src.models import LSTMXGBoostModel
from src.utils import normalize_data, time_series_split

print('All imports successful!')
print(f'TensorFlow version: {tf.__version__}')

## Step 3: Fetch Data


In [ ]:
print('Fetching BTCUSDT 15m data...')
fetcher = CryptoDataFetcher()
btc_15m = fetcher.fetch_symbol_timeframe('BTCUSDT', '15m')
print(f'Data shape: {btc_15m.shape}')
print(f'Date range: {btc_15m.index.min()} to {btc_15m.index.max()}')

## Step 4: Apply ZigZag and Features


In [ ]:
print('Applying ZigZag...')
zigzag = ZigZagIndicator(depth=12, deviation=5, backstep=2)
btc_15m = zigzag.label_kbars(btc_15m)

print('Label distribution:')
for label_id, count in btc_15m['zigzag_label'].value_counts().sort_index().items():
    print(f'  {zigzag.get_label_name(label_id)}: {count}')

print('\nApplying feature engineering...')
fe = FeatureEngineer(lookback_periods=[5, 10, 20, 50, 200])
btc_15m = fe.calculate_all_features(btc_15m)

feature_cols = fe.get_feature_columns(btc_15m)
btc_15m[feature_cols] = btc_15m[feature_cols].fillna(method='ffill').fillna(0)

print(f'Total features: {len(feature_cols)}')

## Step 5: Prepare Training Data


In [ ]:
# Select important features and limit data size for GPU memory
print('Preparing data for training...')

# Use top 40 features
selected_features = feature_cols[:40]

# Time series split
train_df, val_df, test_df = time_series_split(btc_15m, train_ratio=0.7, validation_ratio=0.15)

# Extract features and labels
X_train = train_df[selected_features].values
y_train = train_df['zigzag_label'].values
X_val = val_df[selected_features].values
y_val = val_df['zigzag_label'].values
X_test = test_df[selected_features].values
y_test = test_df['zigzag_label'].values

print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')

# Normalize
print('Normalizing...')
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
X_test = X_test.astype(np.float32)

mean = X_train.mean(axis=0)
std = X_train.std(axis=0) + 1e-8

X_train = (X_train - mean) / std
X_val = (X_val - mean) / std
X_test = (X_test - mean) / std

print('Data normalized')

## Step 6: Create Sequences


In [ ]:
def create_sequences(X, y, timesteps=20):
    X_seq, y_seq = [], []
    for i in range(len(X) - timesteps):
        X_seq.append(X[i:(i + timesteps)])
        y_seq.append(y[i + timesteps])
    return np.array(X_seq, dtype=np.float32), np.array(y_seq)

print('Creating sequences (timesteps=20)...')
X_train_seq, y_train_seq = create_sequences(X_train, y_train, timesteps=20)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, timesteps=20)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, timesteps=20)

print(f'Train sequences: {X_train_seq.shape}')
print(f'Val sequences: {X_val_seq.shape}')
print(f'Test sequences: {X_test_seq.shape}')

# Check class distribution
unique, counts = np.unique(y_train_seq, return_counts=True)
print(f'\nClass distribution in training:')
for u, c in zip(unique, counts):
    print(f'  Class {u}: {c} ({100*c/len(y_train_seq):.1f}%)')

## Step 7: Calculate Class Weights


In [ ]:
# Calculate class weights to handle imbalanced data
from sklearn.utils.class_weight import compute_class_weight

print('Calculating class weights...')

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_seq), y=y_train_seq)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}

print('Class weights:')
for cls, weight in class_weight_dict.items():
    print(f'  Class {cls}: {weight:.4f}')

## Step 8: Build and Train LSTM Model


In [ ]:
print('Building LSTM model...')

model = keras.Sequential([
    keras.layers.LSTM(128, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(64, return_sequences=False),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(5, activation='softmax')  # 5 classes: NO_SIGNAL, HH, HL, LH, LL
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
print('Starting training with class weights on GPU...')

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=200,
    batch_size=64,
    class_weight=class_weight_dict,  # Use class weights
    callbacks=[early_stop],
    verbose=1
)

print('\nTraining complete!')

## Step 9: Evaluate Model


In [ ]:
# Training accuracy
train_loss, train_acc = model.evaluate(X_train_seq, y_train_seq, verbose=0)
print(f'Train Accuracy: {train_acc:.4f}')

# Validation accuracy
val_loss, val_acc = model.evaluate(X_val_seq, y_val_seq, verbose=0)
print(f'Val Accuracy: {val_acc:.4f}')

# Test accuracy
test_loss, test_acc = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'Test Accuracy: {test_acc:.4f}')

# Predictions
y_pred = model.predict(X_test_seq, verbose=0)
y_pred_labels = np.argmax(y_pred, axis=1)

# Classification metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

print(f'\nPrecision: {precision_score(y_test_seq, y_pred_labels, average="weighted", zero_division=0):.4f}')
print(f'Recall: {recall_score(y_test_seq, y_pred_labels, average="weighted", zero_division=0):.4f}')
print(f'F1-Score: {f1_score(y_test_seq, y_pred_labels, average="weighted", zero_division=0):.4f}')

print(f'\nConfusion Matrix:')
cm = confusion_matrix(y_test_seq, y_pred_labels)
print(cm)

print(f'\nClassification Report:')
print(classification_report(y_test_seq, y_pred_labels, zero_division=0))

## Step 10: Save Model


In [ ]:
# Save to Google Drive
model_path = '/content/drive/MyDrive/crypto-zigzag-ml/models/lstm_model_balanced.h5'
model.save(model_path)
print(f'Model saved to: {model_path}')

# Save normalization parameters
import pickle
norm_params = {'mean': mean, 'std': std}
params_path = '/content/drive/MyDrive/crypto-zigzag-ml/models/norm_params_balanced.pkl'
with open(params_path, 'wb') as f:
    pickle.dump(norm_params, f)
print(f'Normalization parameters saved to: {params_path}')

# Save training history
history_dict = {
    'train_loss': history.history['loss'],
    'val_loss': history.history['val_loss'],
    'train_acc': history.history['accuracy'],
    'val_acc': history.history['val_accuracy']
}
history_path = '/content/drive/MyDrive/crypto-zigzag-ml/models/training_history.pkl'
with open(history_path, 'wb') as f:
    pickle.dump(history_dict, f)
print(f'Training history saved to: {history_path}')

## Done!

Your trained balanced LSTM model is ready. Download it from Google Drive and use it for predictions!

Models saved:
- lstm_model_balanced.h5 (the trained model)
- norm_params_balanced.pkl (normalization parameters)
- training_history.pkl (training metrics)
